# TESTI: 'total_installed_price' esikäsittely

Koodi testaa `preprocessor.py` esikäsittelijän toiminnan sarakkeelle 'total_installed_price'. Testi ajetaan kahdella varsinaisella modella: 'minimal' ja 'common'. Vanha 'stand_alone_PV' on korvattu hyödyntämällä uutta saraketta 'battery_storage', joka ilmaisee akun kuulumista järjestelmään.

Lisänä huomioitu lähinnä 'Tesla Energy' asennusyrityksen vaikutus 'price_per_kW' välillä [5100, 5200], jossa on hintatehosuhdejakauman korkein piikki.

## Mode: 'minimal'

In [1]:
import pandas as pd
import numpy as np

import importlib.util
import sys
file_path = "../Tehtava-03/preprocessor.py"; module_name = "preprocessor"; spec = importlib.util.spec_from_file_location(module_name, file_path)
esik = importlib.util.module_from_spec(spec); sys.modules[module_name] = esik; spec.loader.exec_module(esik)


cols = ['total_installed_price', 'price_per_kW', 'installer_name', 'battery_storage']

df = esik.esik(cols, 'minimal')
df.describe(percentiles=[.2, .5, .8])

,total_installed_price,price_per_kW,battery_storage
count,8.143640e+05,814322.000000,1000000.000000
mean,3.824708e+04,4572.313062,0.015449
std,1.919481e+05,2391.154643,0.123330
min,1.000000e-02,0.000019,0.000000
20%,1.464320e+04,3349.720113,0.000000
50%,2.404256e+04,4470.211555,0.000000
80%,3.725029e+04,5383.672555,0.000000
max,6.150719e+07,823842.151675,1.000000


In [2]:
df = df[(df['price_per_kW'] <= 5200) & (df['price_per_kW'] >= 5100)]
df['installer_name'].value_counts().head(5)

Tesla Energy              74781
Vivint Solar Developer     1182
Sunpower Capital            430
Verengo                     415
Petersen Dean               358
Name: installer_name, dtype: int64

## Mode: 'common'

In [3]:
df = esik.esik(cols, 'common')
df.describe(percentiles=[.2, .5, .8])

,total_installed_price,price_per_kW,battery_storage
count,7.601830e+05,640335.000000,1000000.000000
mean,3.866728e+04,4626.638018,0.015449
std,1.932668e+05,2273.649241,0.123330
min,1.000000e-02,1000.000000,0.000000
20%,1.500100e+04,3472.222222,0.000000
50%,2.443190e+04,4499.183673,0.000000
80%,3.744000e+04,5259.938838,0.000000
max,6.150719e+07,823842.151675,1.000000


In [4]:
df = df[(df['price_per_kW'] <= 5200) & (df['price_per_kW'] >= 5100)]
df['installer_name'].value_counts().head(5)

Tesla Energy              73316
Vivint Solar Developer      989
Sunpower Capital            409
Verengo                     393
Petersen Dean               327
Name: installer_name, dtype: int64

## Mode: 'stand_alone_PV'

Tässä tehdään 'stand_alone_PV' suodatus vasta esikäsittelijä funktion jälkeen, koska päivityksen myötä vanha mode poistettiin käytöstä. Uudessa versiossa käytettävä mode on 'common'.

In [5]:
df = esik.esik(cols, 'common')
df = df[(df['battery_storage'] == 0)]
df.describe(percentiles=[.2, .5, .8])

,total_installed_price,price_per_kW,battery_storage
count,7.450100e+05,631646.000000,984551.0
mean,3.831658e+04,4621.127393,0.0
std,1.790540e+05,2274.753643,0.0
min,1.000000e-02,1000.000000,0.0
20%,1.500000e+04,3464.000000,0.0
50%,2.430000e+04,4487.179487,0.0
80%,3.727300e+04,5251.536615,0.0
max,2.000000e+07,823842.151675,0.0


In [6]:
df = df[(df['price_per_kW'] <= 5200) & (df['price_per_kW'] >= 5100)]
df['installer_name'].value_counts().head(5)

Tesla Energy              71286
Vivint Solar Developer      989
Sunpower Capital            409
Verengo                     393
Petersen Dean               308
Name: installer_name, dtype: int64

## Mode: 'paired_PV_and_storage'

Edellisen vastapainoksi vain akkujärjestelmät uutena lisänä.

In [7]:
df = esik.esik(cols, 'common')
df = df[(df['battery_storage'] == 1)]
df.describe(percentiles=[.2, .5, .8])

,total_installed_price,price_per_kW,battery_storage
count,1.517300e+04,8689.000000,15449.0
mean,5.588729e+04,5027.232370,1.0
std,5.448829e+05,2154.568800,0.0
min,1.000000e+00,1000.000000,1.0
20%,2.050000e+04,4010.698396,1.0
50%,3.130100e+04,5120.000000,1.0
80%,4.407510e+04,5556.050480,1.0
max,6.150719e+07,126355.214717,1.0


In [8]:
df = df[(df['price_per_kW'] <= 5200) & (df['price_per_kW'] >= 5100)]
df['installer_name'].value_counts().head(5)

Tesla Energy            2030
Rec Solarmmercialrp       54
Sunrun                    51
Petersen Dean             19
Sullivan Solar Power      16
Name: installer_name, dtype: int64